In [1]:
'''
normal 0
adenocarcinoma 1
lung_cancer 2
lung_carcinoid_tumor 3
small_cell_lung_carcinoma 4
squamous_cell_carcinoma 5
squamous_cell_lung_carcinoma 5
'''

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql import Row
import pandas as pd

In [3]:
cancer_Data = pd.read_csv('/dbfs/FileStore/tables/Labeled_Cancer_Data.csv')
feature_List = cancer_Data.pop('Gene_Symbol')
cancer_Data = cancer_Data.transpose().dropna()
subject_Names = list(cancer_Data.index)
cancer_Train = []
for i in subject_Names:
  cancer_Train.append(Row(label = cancer_Data.loc[i][0], features = Vectors.dense(cancer_Data.loc[i][1:])))
train_Feat = sc.parallelize(cancer_Train).toDF()

/databricks/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (363,726) have mixed types. Specify dtype option on import or set low_memory=False.
 interactivity=interactivity, compiler=compiler, result=result)

In [4]:
model = RandomForestClassifier(numTrees= 100)
model.fit(train_Feat)
feat_Import = model.featureImportances

In [5]:
feature_Data = pd.read_csv('/dbfs/FileStore/tables/Signficant_Features.txt', header=-1)
feature_Data.columns = ['Features','Z-scores']
cancer_Data.columns = feature_List
sig_Features = pd.Index(feature_Data['Features'])
sig_Features = sig_Features.insert(0,'Label')
sig_Data = cancer_Data.loc[:,sig_Features]

In [6]:
sig_Data = sig_Data.sample(frac=1).reset_index(drop=True)
sig_Data.columns = range(sig_Data.shape[1])
cutoff = int(sig_Data.shape[0] * 0.8)
train_Data = []
test_Data = []
for i in range(sig_Data.shape[0]):
  if (i <= cutoff):
    train_Data.append(Row(label = sig_Data.loc[i][0], features = Vectors.dense(sig_Data.loc[i][1:])))
  else:
    test_Data.append(Row(label = sig_Data.loc[i][0], features = Vectors.dense(sig_Data.loc[i][1:])))
train_Df = sc.parallelize(train_Data).toDF()
test_Df = sc.parallelize(test_Data).toDF()

In [7]:
model = RandomForestClassifier(numTrees= 100)
model.fit(train_Df)
predictions = model.transform(test_Df)

Out[56]: RandomForestClassificationModel (uid=RandomForestClassifier_6cfa858e864d) with 100 trees

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-449530349819475> in <module> 
 ----> 1 model . transform ( ) 

 AttributeError : 'RandomForestClassifier' object has no attribute 'transform'